# Inferencia Modelo Mistral Google Colab

**Autor:** Guillermo Gallego Reina

**Fecha:** 14 de Mayo de 2024

**Introducción**

---

En el anterior notebook se explicarón las diferentes etapas para llevar acabo el ajuste de modelo de Mistral dando como resultado un modelo ajustado llamado [mistralAI_recetascocina](https://huggingface.co/guillergalre/mistralAI_recetascocina). Como vimos, este modelo esta ajustado para una dataset de recetas colombinas.

Se explicarán las diferentes etapas para llevar a cabo el proceso de validación entre los diferentes modelos. Estas consistirán en realizar un petición al modelo ajustado y no ajustado comprobando la respuesta y tiempo.
**#LoRA**, **#GenAI**, **#LLM**, **#Mistral**, **#GPU**, **#QLoRA**, **#GoogleColab**.

---

**Tareas:**

1. **[Importar Librerias](#1)**
   - Librerias necesarias para Google Colab.

2. **[Preparación Entorno Hugging Face](#2)**
   - Realizamos la configuracion necesaria para poder acceder a Hugging Face.

3. **[Carga del Modelo](#3)**
   - 3.1 [Cargamos el modelo ajustado: mistralAI_recetascocina](#4)
   - 3.2 [Cargamos el modelo base: Mistral-7B-Instruct-v0.1](#5)
    
4. **[Inferencia](#6)**
    - 4.1 [Preprocesado de acuerdo a la plantilla](#7)
    - 4.2 [Inferencia Modelo Ajustado](#8)
    - 4.3 [Inferencia Modelo Base](#9)

4. **[Conclusiones](#10)**
   - Se obtienen las siguientes conclusiones.


**Entorno de Trabajo:**
   - Se utilizará **Google Colab** por su fácil acceso a GPU y TPU de alto rendimiento, lo que permite realizar tareas de ciencia de datos de manera eficiente y entrenar LLM.

---

<a id="1"></a> 
# 1. Importar Librerias

___

Dado que vamos hacer uso de **Google Colab** es necesario importar librerías especifícas para llevar a cabo el Fine-Tunning.

___

In [1]:
!pip install bitsandbytes transformers peft accelerate
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)

<a id="2"></a> 
## 2. Preparación de Entorno Hugging Face

___

Dado que el entorno sobre el que estamos trabajando es **GoogleColab** es necesario tener acceso en Hugging Face. Para ello deberemos:

* **Crear una cuenta en Hugging Face**
* **Crear un Token en Hugging Face con permisos de **Escritura****
* **Crear sobre GoogleColab la variable HF_TOKEN con el token creado**

En algunas ocasiones no es suficiente crear el **HF_TOKEN**, para prevenir el error se indica un comando para poder logearse directamente con el Token creado.

___

In [2]:
#Para acceder al Token
#Creamos el Token con Write Role
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

<a id="3"></a> 
## 3. Carga del modelo

___

Realizada las configuraciones necesarias para acceder a Hugging Face, ya podemos realizar la carga de los diferentes modelos. Para la carga al igual que explicamos en el anterior notebook vamos a cargalo con un quantificación de 4 por simplicidad de memoria y comlejidad dado las limitaciones de **Google Colab**.

___

<a id="4"></a> 
### 3.1 Carga del modelo ajustado

___

Cargamos el modelo ajustado por los datos [recetas-cocina](https://huggingface.co/datasets/somosnlp/recetas-cocina?_sm_vck=ZbFKqrNsMLPHD4S5QDj4VD7sVRsvTLFksfWTZZq6Tktqr4fbVjrP) llamado [mistralAI_recetascocina](https://huggingface.co/guillergalre/mistralAI_recetascocina).

El procedimiento es muy similar al notebook de Fine-Tunning dado que lo tenemos que cargar con quantificación 4 y deberemos cargar los Tokens y el propio modelo.

___

In [3]:
from transformers import AutoTokenizer
import torch

#Modelo Preentrenado
new_model = "guillergalre/mistralAI_recetascocina"
tokenizerrece = AutoTokenizer.from_pretrained(new_model)

tokenizerrece.pad_token = tokenizerrece.unk_token
tokenizerrece.padding_side = "right"

from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

compute_dtype = getattr(torch, "float16")
use_4bit = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

device_map = "auto"
modelrecetas = AutoModelForCausalLM.from_pretrained(
    new_model,
    quantization_config=bnb_config,  # loading model in 4-bit
    device_map=device_map, # to use max gpu resources if exist
)

#Configure the pad token in the model
modelrecetas.config.pad_token_id = tokenizerrece.pad_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<a id="5"></a> 
### 3.2 Carga del modelo base

___

Cargamos el modelo base [Mistral-7B-Instruct-v0.1](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1).

El procedimiento es muy similar al notebook de Fine-Tunning dado que lo tenemos que cargar con quantificación 4 y deberemos cargar los Tokens y el propio modelo.

___

In [3]:
#Modelo MIstral
from transformers import AutoTokenizer
import torch
mistral = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizermistral = AutoTokenizer.from_pretrained(mistral)

tokenizermistral.pad_token = tokenizermistral.unk_token
tokenizermistral.padding_side = "right"

from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

compute_dtype = getattr(torch, "float16")
use_4bit = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

device_map = "auto"
modelmistral = AutoModelForCausalLM.from_pretrained(
    mistral,
    quantization_config=bnb_config,  # loading model in 4-bit
    device_map=device_map # to use max gpu resources if exist
)

#Configure the pad token in the model
modelmistral.config.pad_token_id = tokenizermistral.pad_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<a id="6"></a> 
## 4. Inferencia

___

La inferencia consisitirá en poner en práctica lo que el modelo ha aprendido durante el ajuste.

___

<a id="7"></a> 
### 4.1 Preprocesado

___

Dado que tenemos que introducir los datos de entrada de acuerdo a la plantilla con la que se ha ajustado/entrenado tenemos que realizar un función que transforme dichas peticiones.

___

In [4]:
def build_prompt(question):
    prompt = '<s>[INST]'+ question +'[/INST]</s>'
    return prompt

<a id="8"></a> 
### 4.2 Inferencia modelo ajustado

___

A partir del modelo cargado realizaremos un pipeline indicandole que la máxima longitud es de 300 palabras. Por otro lado llamaremos a la función creada previamente para preparar la petición de acuerdo a la planilla.

Uno de los parámetros importates en la configuración del pipeline será la task, en nuestro caso tendremos que indicar que genere un texto a partir de la petición indicadada. Para ello el parámetro será **text-generation**.

___

In [6]:
import time
start_time = time.time()

pipe = pipeline(task="text-generation", model=modelrecetas, tokenizer=tokenizerrece, max_length=3000)

question = "Respondeme a la pregunta haciendo una lista: ¿Como harias un Sudado de Pollo?"
prompt = build_prompt(question)

result = pipe(question)

print(result[0]['generated_text'])
print("--- %s seconds ---" % (time.time() - start_time))

Respondeme a la pregunta haciendo una lista: ¿Como harias un Sudado de Pollo? 1. Poner en una olla grande el agua, el pollo, la cebolla, el ajo, el pimentón, el orégano, el comino, la sal y la pimienta. 2. Llevar a ebullición y dejar cocer durante 1 hora. 3. Retirar del fuego y dejar enfriar. 4. Desmenuzar el pollo y mezclar con la salsa. 5. Servir en una fuente y espolvorear con perejil.
--- 20.701589584350586 seconds ---


<a id="9"></a> 
### 4.3 Inferencia modelo base

___

A partir del modelo base realizaremos un pipeline indicandole que la máxima longitud es de 300 palabras. Por otro lado llamaremos a la función creada previamente para preparar la petición de acuerdo a la planilla.

Uno de los parámetros importates en la configuración del pipeline será la task, en nuestro caso tendremos que indicar que genere un texto a partir de la petición indicadada. Para ello el parámetro será **text-generation**.
___

In [5]:
import time
start_time = time.time()

pipe = pipeline(task="text-generation", model=modelmistral, tokenizer=tokenizermistral, max_length=3000)

question = "Respondeme a la pregunta haciendo una lista: ¿Como harias un Sudado de Pollo?"
prompt = build_prompt(question)

result = pipe(question)

print(result[0]['generated_text'])
print("--- %s seconds ---" % (time.time() - start_time))

Respondeme a la pregunta haciendo una lista: ¿Como harias un Sudado de Pollo?

1. En una olla grande, agregue 2 cucharadas de aceite de oliva y caliente sobre fuego mediano.
2. Agregue 1 cucharada de pimienta, 1 cucharada de sal y 1 cucharada de pimienta negra.
3. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
4. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
5. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
6. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
7. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
8. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
9. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
10. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
11. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
12. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
13. Agregue 1 cucharada de pimienta negra y 1 cucharada de sal.
14. Agregue 1 cucharada de pimienta negra

<a id="10"></a> 
## 5. Conclusiones

A pesar que la muestra sobre la que ajustamos el modelo es pequeña (~5K) podemos comprobar que el modelo base no es capaz de devolver una solucción razonable en coparación con el modelo ajustado.

Podemos decir que el experimento es **SATISFACTORIO**.